<a href="https://colab.research.google.com/github/W2NJL/CSC8490/blob/main/CSC8490project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up the Oracle client - as neatly prepared by Prof. Frein Esq.

! pip install cx_Oracle

! ls -l oracle-instantclient*-basiclite-*.rpm || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! ls -l oracle-instantclient*-sqlplus-*.rpm   || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

! sudo apt-get install alien libaio1

! sudo alien -i oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! sudo alien -i oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

%env ORACLE_HOME=/usr/lib/oracle/19.3/client64

%env LD_LIBRARY_PATH=/usr/lib/oracle/19.3/client64/lib

print('Setup finished!')

     |████████████████████████████████| 757kB 5.8MB/s 
ls: cannot access 'oracle-instantclient*-basiclite-*.rpm': No such file or directory
--2020-11-21 02:14:32--  https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm
Resolving yum.oracle.com (yum.oracle.com)... 184.24.37.92
Connecting to yum.oracle.com (yum.oracle.com)|184.24.37.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28553496 (27M) [application/x-rpm]
Saving to: ‘oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm’

oracle-instantclien 100%[===================>]  27.23M  84.1MB/s    in 0.3s    

2020-11-21 02:14:32 (84.1 MB/s) - ‘oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm’ saved [28553496/28553496]

ls: cannot access 'oracle-instantclient*-sqlplus-*.rpm': No such file or directory
--2020-11-21 02:14:32--  https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/

In [ ]:
# connect into Nick's Oracle DB instance on Prof. Frein's cloud server
import numpy as np
import pandas as pd
import cx_Oracle

CONN_INFO = {
 'host': '45.79.135.253',
 'port': '1521',
 'user': 'nlanga01',
 'psw': '00700368',
 'service': 'xe',
}
CONN_STR = '{user}/{psw}@{host}:{port}/{service}'.format(**CONN_INFO)
connection = cx_Oracle.connect(CONN_STR)

cursor = connection.cursor()

print(connection.version)

18.4.0.0.0


In [ ]:
# calling a stored procedure
# run the stored procedure setup in the slides first
# set up a variable to accept the procedure output
result = cursor.var(cx_Oracle.NUMBER)
val = input("Enter the callletters of the station you wish to find the distance in miles from Villanova University: ") 
# calling the procedure
cursor.callproc("FINDDISTANCE", ["Villanova", val, result])
print("Distance from Villanova in miles to " + val + " is: ")
print(result)

Enter the callletters of the station you wish to find the distance in miles from Villanova University: WISX
Distance from Villanova in miles to WISX is: 
<cx_Oracle.Var of type DB_TYPE_NUMBER with value 9.0>


In [ ]:
# run a SELECT and print results
for result in cursor.execute("select * from DXPEDITIONLOGS"):
    print(result)

('WFRB', 'Deep Creek Lake DX', 22, 'Tr')
('WLVV', 'Deep Creek Lake DX', 24, 'Tr')
('W204BA', 'Deep Creek Lake DX', 10, 'Tr')
('WFRJ', 'Deep Creek Lake DX', 60, 'Tr')
('WQED', 'Deep Creek Lake DX', 70, 'Tr')
('WVDS', 'Deep Creek Lake DX', 25, 'Tr')
('WQEJ', 'Deep Creek Lake DX', 60, 'Tr')
('WVNP', 'Deep Creek Lake DX', 79, 'Tr')
('WAIJ', 'Deep Creek Lake DX', 17, 'Tr')
('WESA', 'Deep Creek Lake DX', 70, 'Tr')
('WVPM', 'Deep Creek Lake DX', 24, 'Tr')
('WOUB', 'Deep Creek Lake DX', 151, 'Tr')
('WKRJ', 'Deep Creek Lake DX', 135, 'Tr')
('WWVU', 'Deep Creek Lake DX', 32, 'Tr')
('WFWM', 'Deep Creek Lake DX', 24, 'Tr')
('WCAL', 'Deep Creek Lake DX', 45, 'Tr')
('WJHT', 'Deep Creek Lake DX', 60, 'Tr')
('WKPL', 'Deep Creek Lake DX', 97, 'Tr')
('WKTQ', 'Deep Creek Lake DX', 12, 'Tr')
('WDJQ', 'Deep Creek Lake DX', 126, 'Tr')
('WGIE', 'Deep Creek Lake DX', 55, 'Tr')
('WLTJ', 'Deep Creek Lake DX', 74, 'Tr')
('WFGM', 'Deep Creek Lake DX', 35, 'Tr')
('WQZS', 'Deep Creek Lake DX', 19, 'Tr')
('WFBY', 'D